In [1]:
import numpy as np              # Python's standard numerical library
from shapely.geometry import Polygon
import matplotlib.pyplot as plt # Python's scientific visualization library
import matplotlib.animation as animation
import pyMRAW as mraw
import pyidi

c:\Users\thijs\anaconda3\lib\site-packages\skimage\util\dtype.py:27: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [2]:
# properties constants and paths
path = "C:/Users/thijs/Documents/GitHub/pyidi/data/"
filename = 'synthetic_data_thijs'
image_width     = 1024
image_height    = 512
amplitude       = min(image_width, image_height)/4
fps             = 40000.0
total_time      = 0.05
number_of_rotations = 0.1
frequency       = number_of_rotations/total_time
total_frames    = int(total_time * fps)

t_vec           = np.linspace(0, total_time, total_frames)
j0              = image_width/2
i0              = image_height/2
a0              = 0.45*image_height #radius of rotating square
width           = 4*np.pi

video_info = {'Date': '2023/7/26',
 'Camera Type': 'FASTCAM SA-Z type 2100K-M-64GB',
 'Record Rate(fps)': fps,
 'Shutter Speed(s)': fps + 2748.0,
 'Total Frame': total_frames,
 'Original Total Frame': total_frames*2,
 'Image Width': image_width,
 'Image Height': image_height,
 'File Format': 'Mraw',
 'EffectiveBit Depth': 12,
 'EffectiveBit Side': 'Higher',
 'Color Bit': 16,
 'Comment Text': ''}

total_frames


2000

In [3]:
# Initialize synthetic data for testing
%matplotlib qt
mraw_syn = np.ones((total_frames, image_height, image_width), dtype=np.uint16)*(2**16-1)
sequential_image_n = 0
still_image = mraw_syn[sequential_image_n]
fig, ax = plt.subplots(figsize=(15, 5))
ax.imshow(still_image, cmap='gray', vmin=0, vmax=2**16-1)

In [11]:
def current_corner_location(i0, j0, f0, f1, a0, t, width):
    """ Rotate a sqaure around fixed point (x0, y0). Outputs the x and y coordinates of corners of the rotated square, based on the time t.
    """
    ic = i0 + a0 * np.sin(2 * np.pi * f0 * t)
    jc = j0 + a0 * np.cos(2 * np.pi * f0 * t)
    
    i = []
    j = []
    for corner in range(4):
        i_local = np.sqrt(2)*width/2 * np.sin(2 * np.pi * f1 * t + corner*np.pi/2 + np.pi/4)
        j_local = np.sqrt(2)*width/2 * np.cos(2 * np.pi * f1 * t + corner*np.pi/2 + np.pi/4)
        i.append(ic + i_local)  
        j.append(jc + j_local)
    return i, j


i = np.zeros((len(t_vec),4))
j = np.zeros((len(t_vec),4))
for it, t in enumerate(t_vec):
    i[it], j[it] = current_corner_location(i0, j0, frequency, 0, a0, t, width)


In [12]:
# Simulate the results
%matplotlib qt
def play_video(i, j, frame_range, interval=30, fig_ax = None, mraw = None, lines = True):
    if fig_ax is None:
        fig, ax = plt.subplots()
    else:
        fig, ax = fig_ax
    # ax.set_xlim(x0-1.2*np.max(x), x0+1.2*np.max(x))
    # ax.set_ylim(y0-1.2*np.max(y), y0+1.2*np.max(y))
    if mraw is not None:
        if isinstance(mraw[0][0], np.ndarray):
            im = ax.imshow(mraw[0], cmap='gray', vmin=0, vmax=2**16-1)
        else:
            im = ax.imshow(mraw, cmap='gray', vmin=0, vmax=2**16-1)
    ax.set_xlim(0, image_width)
    ax.set_ylim(image_height,0)
    ax.set_aspect('equal')
    if lines:
        box, = plt.plot(np.append(j[0],j[0][0]), np.append(i[0],i[0][0]), 'r-')
        plt.plot(j0, i0, 'b*')
    text = ax.text(0.95, 0.05, '', transform=ax.transAxes, color='black', ha='right', va='bottom')
    def update(it):
        if mraw is not None:
            if isinstance(mraw[0][0], np.ndarray):
                im.set_data(mraw[it])
                if not lines:
                    return im
        if lines:
            box.set_data(np.append(j[it],j[it][0]), np.append(i[it],i[it][0]))
            text.set_text(f'Frame {it}')
            return im, box, text
        return im, box, text

    
    ani = animation.FuncAnimation(fig, update, frames=frame_range, interval=interval) #
    plt.show()
    return ani

ani = play_video(i, j , range(len(t_vec)), interval=1, mraw = still_image) #fig_ax = (fig, ax)


In [13]:
def find_intersecting_area(pixel_i, pixel_j, i, j):
    """ Find the area of the intersection of the pixel found at (pixel_i, pixel_j) and a square of which the corners are define in x and y."""
    
    pixel = Polygon([(pixel_j - 0.5, pixel_i - 0.5), (pixel_j + 0.5, pixel_i-0.5), (pixel_j+0.5, pixel_i+0.5), (pixel_j-0.5, pixel_i+0.5)])
    rotating_square = Polygon([(j[0], i[0]), (j[1], i[1]), (j[2], i[2]), (j[3], i[3])])
    try:
        intersection_area = rotating_square.intersection(pixel).area       
    except:
        x0, y0 = rotating_square.exterior.xy
        x1, y1 = pixel.exterior.xy
        fig, ax = plt.subplots()
        plt.plot(x0, y0,'r')
        plt.plot(x1, y1,'b')
        ax.set_aspect('equal')
        plt.show()
        intersection_area = 0.0
        del fig, ax
    return intersection_area

find_intersecting_area(np.mean(i[0])-np.round(width/2), np.mean(j[0])-np.round(width/2), i[0], j[0])
# find_intersecting_area(np.mean(j[0])+a0, np.mean(i[0]), j[0], i[0])

0.6133792253819718

In [14]:
%matplotlib qt
nz = 0
roi_size = np.ceil(np.sqrt(2)*width/2).astype(int)
for t_it, (ic, jc) in enumerate(zip(i, j)):
    pix_ic = np.round(np.mean(ic)).astype(int)
    pix_jc = np.round(np.mean(jc)).astype(int)
    for pix_j in range(pix_jc - roi_size, pix_jc + roi_size):
        for pix_i in range(pix_ic - roi_size, pix_ic + roi_size):
            a = find_intersecting_area(pix_i, pix_j, ic, jc)
            if a>1:
                print('something is wrong')
            elif a>0:
                nz+=1
            mraw_syn[t_it, pix_i, pix_j]  = (2**16-1) - np.round(a*(2**16-1))
            
nz

c:\Users\thijs\anaconda3\lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


368350

In [15]:
ani = play_video(i, j , range(len(t_vec)), interval=1, mraw = mraw_syn, lines=False)

In [ ]:
# Save synthetic data to mraw file
mraw.save_mraw(mraw_syn, path + filename + '_with_rotation.mraw', info_dict=video_info)
del mraw_syn
video = pyidi.pyIDI(path + filename +'.cih')
mraw_syn = video.mraw
# mraw_syn

In [16]:
writer = animation.FFMpegWriter(fps=60)
ani.save('anim/'+filename + '_without_rotation_anim_'+'.mp4', writer=writer)